# Displaying strike and dip symbols on a map using folium
### O. Kaufmann, 2018.

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
import folium

<div class="alert alert-block alert-danger">
**First start a simple http server to serve the symbols**  
Open a new terminal move to the work directory and run the following command  
`python3 -m http.server 3333 --bind 127.0.0.1`
</div>

### Create a pandas dataframe with the position of the symbols given in the projected coordinate reference system of the map and the strike and dip angles given in degrees using the rigth hand side rule

In [3]:
df = pd.DataFrame([[120150, 122145,120, 0.], 
                   [121450, 122255, 110, 90.], 
                   [120874, 123059, 0, 60],
                   [121400, 123500, 245, 10]], 
                  columns=['x', 'y', 'strike', 'dip'])

In [4]:
gs = gpd.GeoSeries(df.apply(lambda row: Point(row['x'], row['y']), 1),crs={'init': 'epsg:31370'})

In [5]:
gs = gpd.GeoSeries(df.apply(lambda row: Point(row['x'], row['y']), 1),crs={'init': 'epsg:32631'})

In [6]:
gdf = gpd.GeoDataFrame(df, geometry=gs)
gdf

,x,y,strike,dip,geometry
0,120150,122145,120,0.0,POINT (120150 122145)
1,121450,122255,110,90.0,POINT (121450 122255)
2,120874,123059,0,60.0,POINT (120874 123059)
3,121400,123500,245,10.0,POINT (121400 123500)


### Create a function to draw a symbol given the position and angles and save this symbol as a png image

In [7]:
def plot_strike_dip(pos, strike, dip, degrees=True, linewidth=40., textsize=20.,
                    color='black', png_fig=None):
    # right hand side rule
    import numpy as np
    
    if degrees:
        strike = strike*np.pi/180.
        dip = dip*np.pi/180.

    dl = 0.4
        
    fig, ax=plt.subplots(figsize=(4,4));
    
    if dip == 0.: # horizontal
        strike = 0.
        horizontal_dip_circle = plt.Circle((0., 0.), 1.0, color=color, fill=False, linewidth=linewidth)
        ax.add_artist(horizontal_dip_circle)
        ax.plot([-np.sin(strike+np.pi/2), np.sin(strike+np.pi/2)], 
                [-np.cos(strike+np.pi/2), np.cos(strike+np.pi/2)], 
                linewidth=linewidth, color=color);
        
    elif dip == np.pi/2.: #vertical
        ax.plot([-np.sin(strike+np.pi/2)*dl, np.sin(strike+np.pi/2)*dl], 
                [-np.cos(strike+np.pi/2)*dl, np.cos(strike+np.pi/2)*dl], 
                linewidth=linewidth, color=color);
        
    else:
        ax.plot([0.0, np.sin(strike+np.pi/2)*dl], 
                [0.0, np.cos(strike+np.pi/2)*dl],
                linewidth=linewidth, color=color);
        txt = ax.text(0., 0., '%02.f°' %(dip*180/np.pi), size=textsize, color=color,
                  horizontalalignment='center', verticalalignment='center');
    
        txt.set_rotation(90-strike*180/np.pi);
        txt.set_position((2.5*np.sin(strike+np.pi/2)*dl, 2.5*np.cos(strike+np.pi/2)*dl))
    
    #plotting STRIKE line which is 2 x radius of 1.0
    ax.plot([-np.cos(np.pi/2.-strike), np.cos(np.pi/2.-strike)], 
            [-np.sin(np.pi/2.-strike), np.sin(np.pi/2.-strike)],
            linewidth=linewidth, color=color);
    
       
    #ax.plot(0., 0., '.c', markersize = 50)
    
    if png_fig is not None:
        ax.axis('off');
        ax.set_position([-1, -1, 1, 1]);
        ax.set_xlim([-1.25,1.25]);
        ax.set_ylim([-1.25,1.25]);
        ax.set_aspect('equal');
        ax.figure.canvas.draw();
        ax.margins(1.);
        ax.tick_params(which='both', direction='in');
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted());
        plt.savefig(png_fig + '.png', dpi=166, bbox_inches=extent, format='png', transparent=True);
        ax.figure.clf()

In [8]:
gdf.apply(lambda row: plot_strike_dip(Point(row['x'], row['y']), row['strike'], row['dip'], textsize=80, color='blue', linewidth=25., png_fig=str(row.name)),1)
for idx, row in gdf.iterrows():
    gdf.loc[idx, 'icon'] = 'http://localhost:3333/' + str(idx) + '.png'
gdf

,x,y,strike,dip,geometry,icon
0,120150,122145,120,0.0,POINT (120150 122145),http://localhost:3333/0.png
1,121450,122255,110,90.0,POINT (121450 122255),http://localhost:3333/1.png
2,120874,123059,0,60.0,POINT (120874 123059),http://localhost:3333/2.png
3,121400,123500,245,10.0,POINT (121400 123500),http://localhost:3333/3.png


<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

In [9]:
gdf.to_crs({'init': 'epsg:4326'}, inplace=True)

In [10]:
center=(gdf.loc[0, 'geometry'].y, gdf.loc[0, 'geometry'].x)
center=(22.3758588074800, 4.107834145216459)
center

(22.37585880748, 4.107834145216459)

In [11]:
from IPython.display import Image
m = folium.Map(center, zoom_start=13)
for idx, row in gdf.iterrows():
        icon_url = row['icon']
        popup = '%03.f° - %02.f°' %(row['strike'], row['dip'])
        icon = folium.features.CustomIcon(icon_url, icon_size=(32, 32))
        marker = folium.Marker([row.geometry.y, row.geometry.x], icon=icon, popup=popup)
        m.add_child(marker)

In [12]:
folium.GeoJson(open('polygon.geojson').read(), name='geojson').add_to(m)

In [13]:
cgdf= gpd.read_file('polygon.geojson')
cgdf.to_crs({'init': 'epsg:4326'}, inplace=True)
cgdf

,geometry
0,POLYGON Z ((4.107834145216459 22.3758588074800...


In [14]:
folium.GeoJson(cgdf.to_json(), style_function=lambda feature: {'color' : 'red',
        'weight' : 1., 'fill': False, 'fillOpacity': 0.5,
        }).add_to(m)

In [15]:
folium.LayerControl().add_to(m)

In [16]:
m